# **Hyperparameter Tuning with MLflow & hyperopt**

In [2]:
# necessary libs
!pip install hyperopt
!pip install mlflow
!pip install pyngrok

In [3]:
# importing them - for ml flow/ml
import os
import argparse
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

#loading libs for the server
import pyngrok
from pyngrok import ngrok, conf
import subprocess
import getpass

In [4]:
# for hyper-parameter finetuning
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [5]:
# Reading the data
data_url = "https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/refs/heads/master/winequality-red.csv"

df = pd.read_csv(data_url)

In [6]:
#Function for performance evaluation
def evaluate(y,pred):
    rmse = np.sqrt(mean_squared_error(y,pred))
    mae = mean_absolute_error(y,pred)
    r2 = r2_score(y,pred)

    return rmse, mae, r2

In [7]:
# dataset prep
train,test = train_test_split(df,random_state=123)
train_x = train.drop(["quality"],axis=1)
test_x = test.drop(["quality"],axis=1)

train_y = train[["quality"]]
test_y = test[["quality"]]

In [9]:
#MLflow config
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI])

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("hyperparameter-experiment")

<Experiment: artifact_location='/content/mlruns/1', creation_time=1747442467281, experiment_id='1', last_update_time=1747442467281, lifecycle_stage='active', name='hyperparameter-experiment', tags={}>

In [12]:
# ngrok set up
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()
port=5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel "https://f90f-34-106-135-176.ngrok-free.app" -> "http://127.0.0.1:5000"


In [13]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "Elasticnet")
        mlflow.log_params(params)

        lr = ElasticNet(**params)
        lr.fit(train_x,train_y)

        pred = lr.predict(test_x)

        rmse,mae,r2 = evaluate(test_y,pred)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae",mae)
        mlflow.log_metric("r2",r2)

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
search_space = { "alpha": hp.loguniform('alpha',0.01,1),
                  "l1_ratio": hp.uniform('l1_ratio',0,1)}

In [15]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

100%|██████████| 10/10 [00:01<00:00,  6.94trial/s, best loss: 0.7179008437120649]


In [16]:
#best model
params= {"alpha":1.0404948264732425,
"copy_X":True,
"fit_intercept":True,
"l1_ratio":0.6823008984208807,
"max_iter":1000,
"normalize":"deprecated",
"positive":False,
"precompute":False,
"random_state":None,
"selection":"cyclic",
"tol":0.0001,
"warm_start":False}

In [19]:
# with auto log
mlflow.sklearn.autolog()
with mlflow.start_run() :
    # Remove the 'normalize' key as it's not a valid argument in recent scikit-learn versions
    if 'normalize' in params:
        del params['normalize']
    lr = ElasticNet(**params)
    lr.fit(train_x,train_y)

    pred = lr.predict(test_x)

    rmse,mae,r2 = evaluate(test_y,pred)

    # The variables alpha and l1_ratio are not defined in this scope.
    # Access them from the params dictionary instead.
    print(f"Elastic net Params: alpha: {params['alpha']}, l1_ratio: {params['l1_ratio']}")
    print(f"Elastic net metric: rmse:{rmse}, mae:{mae},r2:{r2}")

Elastic net Params: alpha: 1.0404948264732425, l1_ratio: 0.6823008984208807
Elastic net metric: rmse:0.7896212045750471, mae:0.6491379913030703,r2:0.019034539467356226
